In [1]:
import findspark
findspark.init('/home/gani/spark-2.4.4-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, concat, col, split

In [3]:
spark = SparkSession.builder.appName('lab220').getOrCreate()

In [4]:
df = spark.read.json('Restaurants_in_Durham_County_NC.json')

In [7]:
df.show(5)
df.head(2)

+----------------+--------------------+--------------------+--------------------+--------------------+
|       datasetid|              fields|            geometry|    record_timestamp|            recordid|
+----------------+--------------------+--------------------+--------------------+--------------------+
|restaurants-data|[, Full-Service R...|[[-78.9573299, 35...|2017-07-13T09:15:...|1644654b953d1802c...|
|restaurants-data|[, Nursing Home, ...|[[-78.8895483, 36...|2017-07-13T09:15:...|93573dbf8c9e799d8...|
|restaurants-data|[, Fast Food Rest...|[[-78.9593263, 35...|2017-07-13T09:15:...|0d274200c7cef50d0...|
|restaurants-data|[, Full-Service R...|[[-78.9060312, 36...|2017-07-13T09:15:...|cf3e0b175a6ebad2a...|
|restaurants-data|[,, [36.0556347, ...|[[-78.9135175, 36...|2017-07-13T09:15:...|e796570677f7c39cc...|
+----------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



[Row(datasetid='restaurants-data', fields=Row(closing_date=None, est_group_desc='Full-Service Restaurant', geolocation=[35.9207272, -78.9573299], hours_of_operation=None, id='56060', insp_freq=4, opening_date='1994-09-01', premise_address1='4711 HOPE VALLEY RD', premise_address2='SUITE 6C', premise_city='DURHAM', premise_name='WEST 94TH ST PUB', premise_phone='(919) 403-0025', premise_state='NC', premise_zip='27707', risk=4, rpt_area_desc='Food Service', seats=60, sewage='3 - Municipal/Community', smoking_allowed='NO', status='ACTIVE', transitional_type_desc='FOOD', type_description='1 - Restaurant', water='5 - Municipal/Community'), geometry=Row(coordinates=[-78.9573299, 35.9207272], type='Point'), record_timestamp='2017-07-13T09:15:31-04:00', recordid='1644654b953d1802c3c941211f61be1f727b2951'),
 Row(datasetid='restaurants-data', fields=Row(closing_date=None, est_group_desc='Nursing Home', geolocation=[36.0467802, -78.8895483], hours_of_operation=None, id='58123', insp_freq=4, openin

In [6]:
df.printSchema()

root
 |-- datasetid: string (nullable = true)
 |-- fields: struct (nullable = true)
 |    |-- closing_date: string (nullable = true)
 |    |-- est_group_desc: string (nullable = true)
 |    |-- geolocation: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- hours_of_operation: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- insp_freq: long (nullable = true)
 |    |-- opening_date: string (nullable = true)
 |    |-- premise_address1: string (nullable = true)
 |    |-- premise_address2: string (nullable = true)
 |    |-- premise_city: string (nullable = true)
 |    |-- premise_name: string (nullable = true)
 |    |-- premise_phone: string (nullable = true)
 |    |-- premise_state: string (nullable = true)
 |    |-- premise_zip: string (nullable = true)
 |    |-- risk: long (nullable = true)
 |    |-- rpt_area_desc: string (nullable = true)
 |    |-- seats: long (nullable = true)
 |    |-- sewage: string (nullable = true)
 |   

In [10]:
df = df.withColumn('county', lit('Durham')) \
        .withColumn('datasetId', col('fields.id')) \
        .withColumn('name', col('fields.premise_name')) \
        .withColumn('address1', col('fields.premise_address1')) \
        .withColumn('address2', col('fields.premise_address2')) \
        .withColumn('city', col('fields.premise_city')) \
        .withColumn('state', col('fields.premise_state')) \
        .withColumn('zip', col('fields.premise_zip')) \
        .withColumn('tel', col('fields.premise_phone')) \
        .withColumn('dateStart', col('fields.opening_date')) \
        .withColumn('dateEnd', col('fields.closing_date')) \
        .withColumn('type', split(col('fields.type_description'), '-').getItem(1)) \
        .withColumn('geoX', col('fields.geolocation').getItem(0)) \
        .withColumn('geoY', col('fields.geolocation').getItem(1))

In [11]:
df = df.withColumn('id', concat(col('state'), lit('-_'), 
                                col('county'), lit('_'), col('datasetId')))

In [12]:
df.show(5)

+---------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------+------+-----+-----+--------------+----------+-------+--------------------+----------+-----------+----------------+
|datasetId|              fields|            geometry|    record_timestamp|            recordid|county|                name|            address1|address2|  city|state|  zip|           tel| dateStart|dateEnd|                type|      geoX|       geoY|              id|
+---------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+--------+------+-----+-----+--------------+----------+-------+--------------------+----------+-----------+----------------+
|    56060|[, Full-Service R...|[[-78.9573299, 35...|2017-07-13T09:15:...|1644654b953d1802c...|Durham|    WEST 94TH ST PUB| 4711 HOPE VALLEY RD|SUITE 6C|DURHAM|   NC|27707|(919) 403-0025|1994-09-0

In [13]:
df.printSchema()

root
 |-- datasetId: string (nullable = true)
 |-- fields: struct (nullable = true)
 |    |-- closing_date: string (nullable = true)
 |    |-- est_group_desc: string (nullable = true)
 |    |-- geolocation: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- hours_of_operation: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- insp_freq: long (nullable = true)
 |    |-- opening_date: string (nullable = true)
 |    |-- premise_address1: string (nullable = true)
 |    |-- premise_address2: string (nullable = true)
 |    |-- premise_city: string (nullable = true)
 |    |-- premise_name: string (nullable = true)
 |    |-- premise_phone: string (nullable = true)
 |    |-- premise_state: string (nullable = true)
 |    |-- premise_zip: string (nullable = true)
 |    |-- risk: long (nullable = true)
 |    |-- rpt_area_desc: string (nullable = true)
 |    |-- seats: long (nullable = true)
 |    |-- sewage: string (nullable = true)
 |   